In [1]:
from elasticsearch_dsl import Search, Q, Range, connections
from elasticsearch.client import Elasticsearch
from tqdm.notebook import tqdm
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import os
import json
import re
import numpy as np
import ast

In [5]:
df = pd.read_csv(f"../data/events/df_events 2025-01-11_21-36-13.csv", index_col=0)

In [ ]:
df.tail()

In [8]:
ES_INDEX = "merged_events"

In [ ]:
connections.create_connection(alias="ProductionEnvironment", hosts="http://ha-proxy-elasticsearch:9200", timeout= None)

In [10]:
search = Search(using="ProductionEnvironment", index="merged_locations")

In [11]:
city_query = (Q("match", address__city="Dresden")   )
name_query = (Q("match", name="Zwinger")   )
name_query = (Q("match", address__streetHouseNumber="Sophienstraße")   )



In [ ]:
filter = search.query(
    city_query & name_query
)
filter.count()

In [ ]:
locs = []

for hit in tqdm(filter.scan(), total = filter.count() ):
    locs.append(hit.to_dict())

In [16]:
def flatten_dict(d, parent_key='', sep='_'):
    """
    Recursively flatten a nested dictionary.
    """
    items = []
    for k, v in d.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            # Recursively flatten nested dictionaries
            items.extend(flatten_dict(v, new_key, sep=sep).items())
        elif isinstance(v, list):
            # Handle lists: convert to string (comma-separated)
            items.append((new_key, ', '.join(map(str, v))))
        else:
            items.append((new_key, v))
    return dict(items)


In [17]:
locs= [flatten_dict(loc) for loc in locs]

In [ ]:
df_locs = pd.DataFrame(locs)

In [ ]:
df_locs.columns

In [21]:
df_locs.dropna(subset=["address_coordinate_lat", "address_coordinate_lon"], inplace=True)

In [ ]:
df_locs[["address_coordinate_lat", "address_coordinate_lon"]].mean(axis=0)